In [1]:
from jupyter_dash import JupyterDash

In [70]:
import jupyter_dash
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

from datetime import datetime as dt
import pandas as pd
import time
import random
import numpy as np
import plotly.express as px

import re
import locale
locale.setlocale(locale.LC_TIME,'')

'fr_FR.UTF-8'

In [146]:
# chargement des données csv
articles = pd.read_json('/home/sid2018-6/projetInterPromo2021/df_deduplicated_v4.json')
articles = articles.dropna(subset=['art_title', 'art_url','art_content'])
articles = articles.reset_index(drop=True)
articles.head()

,art_auth,art_content,art_content_html,art_extract_datetime,art_id,art_lang,art_tag,art_title,art_url,src_img,src_name,src_type,src_url
0,None,La FNCDG et l’ANDCDG ont publié en septembre l...,"<p style=""text-align: justify;"">La FNCDG et l’...",22 septembre 2020,1,fr,None,9ème édition du Panorama de l’emploi territorial,http://fncdg.com/9eme-edition-du-panorama-de-l...,http://fncdg.com/wp-content/uploads/2020/09/im...,FNCDG,xpath_source,http://fncdg.com/actualites/
1,None,Malgré la levée des mesures de confinement le ...,"<p style=""text-align: justify;"">Malgré la levé...",17 mars 2020,2,fr,None,ACTUALITÉS FNCDG / COVID19,http://fncdg.com/actualites-covid19/,http://fncdg.com/wp-content/uploads/2020/03/co...,FNCDG,xpath_source,http://fncdg.com/actualites/
2,None,Quels étaient les objectifs poursuivis par le ...,"<p style=""text-align: justify;""><strong>Quels ...",24 octobre 2019,25,fr,None,"Interview de M. Olivier DUSSOPT, Secretaire d’...",http://fncdg.com/interview-de-m-olivier-dussop...,http://fncdg.com/wp-content/uploads/2019/10/in...,FNCDG,xpath_source,http://fncdg.com/actualites/
3,None,"La journée thématique, qui aura lieu durant le...","<p style=""text-align: justify;""><strong>La jo...",31 mai 2017,27,fr,None,Journée Thématique FNCDG « Les services de san...,http://fncdg.com/journee-thematique-fncdg-les-...,http://fncdg.com/wp-content/uploads/2017/05/pu...,FNCDG,xpath_source,http://fncdg.com/actualites/
4,None,La 1ère journée thématique en région sur le th...,"<p style=""text-align: justify;"">La 1<sup>ère</...",13 mars 2017,28,fr,None,Journée Thématique FNCDG « Vers de nouveaux mo...,http://fncdg.com/journee-thematique-fncdg-vers...,http://fncdg.com/wp-content/uploads/2017/03/Sa...,FNCDG,xpath_source,http://fncdg.com/actualites/


In [237]:
# dt.strptime(articles['art_extract_datetime'][0], '%d %B %Y').strftime('%d/%m/%Y')
def date_propre(extract_datetime):
    chaine = str(extract_datetime)
    mois_fr = ['janvier','février','mars','avril','mai','juin','juillet','août',
               'septembre','octobre','novembre','décembre']
    m_fr = ['janv','fév','mars','avril','mai','juin','juil','aout','sept','oct','nov','déc']
    c1 = re.compile(r'\d{2}/\d{2}/\d{2}')
    c2 = re.compile(r'\d+ \w+ \d{4}')
    c3 = re.compile(r'\w+ \d+, \d{4}')
    c4 = re.compile(r'\w+  \d{4}')
    c5 = re.compile(r'\d+.\d+.\d+')
    c6 = re.compile(r'\w+.\d+')
    
    
    if c1.match(chaine) is not None:
        return extract_datetime
    
    elif c2.match(chaine) is not None:
        try:
            locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')
            return(dt.strptime(chaine, '%d %B %Y').strftime('%d/%m/%Y'))
        except:
            locale.setlocale(locale.LC_ALL, 'en_GB.UTF-8')
            return(dt.strptime(chaine, '%d %B %Y').strftime('%d/%m/%Y'))
        
    elif c3.match(chaine) is not None:
        chaine = chaine.split()
        if len(chaine[1])==2:
            chaine[1] = '0'+str(chaine[1])
        if chaine[0].lower() in mois_fr:
            locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')
            return(dt.strptime(' '.join(chaine), '%B %d, %Y').strftime('%d/%m/%Y'))
        elif len(chaine[0]) == 4 or 'é' in chaine[0]:
            chaine[0]=chaine[0]+'.'
            locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')
            return(dt.strptime(' '.join(chaine), '%b %d, %Y').strftime('%d/%m/%Y'))
        elif len(chaine[0]) >=5:
            locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')
            return(dt.strptime(' '.join(chaine), '%B %d, %Y').strftime('%d/%m/%Y'))
        else:
            locale.setlocale(locale.LC_ALL, 'en_GB.UTF-8')
            return(dt.strptime(' '.join(chaine), '%b %d, %Y').strftime('%d/%m/%Y'))
    
    elif c4.match(chaine) is not None:
        locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')
        return(dt.strptime(chaine, '%B %Y').strftime('%d/%m/%Y'))
    
    elif c5.match(chaine) is not None:
        chaine = chaine.strip()
        if len(chaine.split('.')[2]) == 4:
            return(dt.strptime(chaine, '%d.%m.%Y').strftime('%d/%m/%Y'))
        else:
            return(dt.strptime(chaine, '%d.%m.%y').strftime('%d/%m/%Y'))
    
    elif c6.match(chaine) is not None:
        chaine = chaine.strip()
        if chaine.split('.')[0].lower() in m_fr:
            locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')
            return(dt.strptime(chaine, '%b%Y').strftime('%d/%m/%Y'))
        else:
            locale.setlocale(locale.LC_ALL, 'en_GB.UTF-8')
            return(dt.strptime(chaine, '%b.%Y').strftime('%d/%m/%Y'))
    
    else:
        if c2.findall(chaine)!=[]:
            return date_propre(c2.findall(chaine)[0])
        elif c1.findall(chaine)!=[]:
            return date_propre(c1.findall(chaine)[0])
        elif c3.findall(chaine)!=[]:
            return date_propre(c3.findall(chaine)[0])
        else:
            return None
    

In [240]:
articles['art_date'] = articles['art_extract_datetime'].apply(date_propre)
articles

,art_auth,art_content,art_content_html,art_extract_datetime,art_id,art_lang,art_tag,art_title,art_url,src_img,src_name,src_type,src_url,art_date
0,None,La FNCDG et l’ANDCDG ont publié en septembre l...,"<p style=""text-align: justify;"">La FNCDG et l’...",22 septembre 2020,1,fr,None,9ème édition du Panorama de l’emploi territorial,http://fncdg.com/9eme-edition-du-panorama-de-l...,http://fncdg.com/wp-content/uploads/2020/09/im...,FNCDG,xpath_source,http://fncdg.com/actualites/,22/09/2020
1,None,Malgré la levée des mesures de confinement le ...,"<p style=""text-align: justify;"">Malgré la levé...",17 mars 2020,2,fr,None,ACTUALITÉS FNCDG / COVID19,http://fncdg.com/actualites-covid19/,http://fncdg.com/wp-content/uploads/2020/03/co...,FNCDG,xpath_source,http://fncdg.com/actualites/,17/03/2020
2,None,Quels étaient les objectifs poursuivis par le ...,"<p style=""text-align: justify;""><strong>Quels ...",24 octobre 2019,25,fr,None,"Interview de M. Olivier DUSSOPT, Secretaire d’...",http://fncdg.com/interview-de-m-olivier-dussop...,http://fncdg.com/wp-content/uploads/2019/10/in...,FNCDG,xpath_source,http://fncdg.com/actualites/,24/10/2019
3,None,"La journée thématique, qui aura lieu durant le...","<p style=""text-align: justify;""><strong>La jo...",31 mai 2017,27,fr,None,Journée Thématique FNCDG « Les services de san...,http://fncdg.com/journee-thematique-fncdg-les-...,http://fncdg.com/wp-content/uploads/2017/05/pu...,FNCDG,xpath_source,http://fncdg.com/actualites/,31/05/2017
4,None,La 1ère journée thématique en région sur le th...,"<p style=""text-align: justify;"">La 1<sup>ère</...",13 mars 2017,28,fr,None,Journée Thématique FNCDG « Vers de nouveaux mo...,http://fncdg.com/journee-thematique-fncdg-vers...,http://fncdg.com/wp-content/uploads/2017/03/Sa...,FNCDG,xpath_source,http://fncdg.com/actualites/,13/03/2017
5,None,L’une des innovations de la loi n°2019-828 du ...,"<p style=""text-align: justify;"">L’une des inno...",22 octobre 2020,30,fr,None,La publication d’un guide d’accompagnement à l...,http://fncdg.com/la-publication-dun-guide-dacc...,http://fncdg.com/wp-content/uploads/2020/10/LG...,FNCDG,xpath_source,http://fncdg.com/actualites/,22/10/2020
6,None,"La FNCDG mène, en collaboration avec d’autres ...","<p style=""text-align: justify;"">La FNCDG mène,...",10 décembre 2020,31,fr,None,La publication d’un guide de sensibilisation a...,http://fncdg.com/la-publication-dun-guide-de-s...,http://fncdg.com/wp-content/uploads/2020/12/im...,FNCDG,xpath_source,http://fncdg.com/actualites/,10/12/2020
7,None,"Créé pour et par les décideurs territoriaux, É...","<p style=""text-align: justify;"">Créé pour et p...",24 février 2017,32,fr,None,Lancement du réseau Étoile,http://fncdg.com/lancement-du-reseau-etoile/,http://fncdg.com/wp-content/uploads/2017/02/re...,FNCDG,xpath_source,http://fncdg.com/actualites/,24/02/2017
8,None,Les décrets n°2017-397 et n°2017-318 du 24 mar...,"<p style=""text-align: justify;"">Les décrets n°...",5 avril 2017,34,fr,None,Le cadre d’emplois des agents de police munici...,http://fncdg.com/le-cadre-demplois-des-agents-...,http://fncdg.com/wp-content/uploads/2017/04/po...,FNCDG,xpath_source,http://fncdg.com/actualites/,05/04/2017
9,None,Une candidate à un examen professionnel organi...,"<p style=""text-align: justify;"">Une candidate ...",6 juillet 2017,35,fr,None,Le Conseil d’Etat confirme la souveraineté des...,http://fncdg.com/le-conseil-detat-confirme-la-...,http://fncdg.com/wp-content/uploads/2017/07/Co...,FNCDG,xpath_source,http://fncdg.com/actualites/,06/07/2017


In [246]:
articles[articles['art_id'].isin([30,25,1])]

,art_auth,art_content,art_content_html,art_extract_datetime,art_id,art_lang,art_tag,art_title,art_url,src_img,src_name,src_type,src_url,art_date
0,None,La FNCDG et l’ANDCDG ont publié en septembre l...,"<p style=""text-align: justify;"">La FNCDG et l’...",22 septembre 2020,1,fr,None,9ème édition du Panorama de l’emploi territorial,http://fncdg.com/9eme-edition-du-panorama-de-l...,http://fncdg.com/wp-content/uploads/2020/09/im...,FNCDG,xpath_source,http://fncdg.com/actualites/,22/09/2020
2,None,Quels étaient les objectifs poursuivis par le ...,"<p style=""text-align: justify;""><strong>Quels ...",24 octobre 2019,25,fr,None,"Interview de M. Olivier DUSSOPT, Secretaire d’...",http://fncdg.com/interview-de-m-olivier-dussop...,http://fncdg.com/wp-content/uploads/2019/10/in...,FNCDG,xpath_source,http://fncdg.com/actualites/,24/10/2019
5,None,L’une des innovations de la loi n°2019-828 du ...,"<p style=""text-align: justify;"">L’une des inno...",22 octobre 2020,30,fr,None,La publication d’un guide d’accompagnement à l...,http://fncdg.com/la-publication-dun-guide-dacc...,http://fncdg.com/wp-content/uploads/2020/10/LG...,FNCDG,xpath_source,http://fncdg.com/actualites/,22/10/2020


In [268]:
# min([time.strftime('%d/%m/%Y', time.gmtime(random.randint(0, int(time.time())))) for x in range(len(articles))])
max(articles['art_date'].dropna())

'31/12/2019'

In [6]:
#Constantes utiles

tab_style = {
    'borderBottom': '1px solid #B80718',
    'padding': '6px',
    'fontWeight': 'bold'
}

tab_select_style = {
    'borderTop': '1px solid #B80718',
    'borderBottom': '1px solid #B80718',
    'backgroundColor': '#B80718',
    'color': 'white',
    'padding': '6px'
}

input_style = {
    'width':'auto'
}

# the style arguments for the sidebar. We use position:fixed and a fixed width
SIDEBAR_STYLE = {
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "2rem 1rem",
    "background-color": "#f8f9fa",
}

CONTENT_STYLE = {
    "padding": "2rem 1rem",
    "width":"auto",
    "background-color": "#f8f9fa",
}

button_style = {"vertical-align": "middle","border-radius": "8px",
                "background-color": "white",
                "color": "black",
                "border": "2px solid #e7e7e7"}

text_style = {"width":"auto","align" : "justify","text-align": "justify"}

In [260]:
#Funtion to create language options for the dropdown
def get_lang_options(articles):
    grp_languages = articles.groupby('art_lang')[["art_title"]].count().reset_index().rename(columns={'art_title':'nb_articles'})
    list_lang=[]
    for lang in range(0,len(grp_languages[['art_lang']])):
        list_lang.append(grp_languages.loc[lang]['art_lang'])
    lang_options=[]
    for n in range(0,len(list_lang)):
        dict={'label':list_lang[n],'value':list_lang[n]}
        lang_options.append(dict)
    return lang_options

#Fonction servant à séparer les différents thèmes fusionnés dans un unique champ.
#Cette fonction est appellée dans extraction.
#Extrait est un champ (string)contenant différents thèmes.
#Index est une liste contenant la position des virgules qui serviront à caster les thèmes.
def separation(extrait,index):
    #Création d'une liste de sortie qui comprendra les thèmes castés.
    t=[]
    #Cette boucle répétera la séparation du champ autant de fois que nécéssaire.
    for i in range(0,len(index)):
        #Si i vaut 0 alors on partira du premier caractère du champ.
        if i==0:
            t.append(extrait[0:index[i]])
        #Aussinon on partira du caractère qui suit la virgule précédente.
        else:
            t.append(extrait[index[i-1]+1:index[i]])
    return t

#Fonction servant à extraire les thèmes, elle s'appuie sur la fonction separation.
#Prend en entrée la liste des thèmes sans doublons.
# def extraction(themes_uniques):
#     #Création de la liste de sortie.
#     themes_finaux=[]
#     #Cette boucle répétera l'extraction des thèmes sur tous les champs de la liste d'entrée.
#     for i in range(0,len(themes_uniques)):
#         #Seul les champs ayant une virgule ont plusieurs thèmes.
#         if ',' in themes_uniques[i]:
#             #Création d'une liste qui regroupe les positions des virgules.
#             index=[]
#             #On sait qu'il y a autant de thèmes que de de virgule+1 (1 virgule sépare 2 mots).
#             for j in range(0,themes_uniques[i].count(',')+1):
#                 #Si j vaut 0 alors on extrait le premier thème, on cherche donc la première virgule.
#                 if j==0:
#                     index.append(themes_uniques[i].find(','))
#                 #Aussinon, pour trouver la virgule suivante, il faut partir du caractère juste après la virgule précédente.
#                 else:
#                     index.append(themes_uniques[i].find(',',index[j-1]+1))
#             #Une fois index remplie, on peut utiliser la fonction séparation.
#             #On rajoutera le résultat à la liste de sortie.
#             themes_finaux.extend(separation(themes_uniques[i],index))
#         else:
#             #Si le champ n'a pas de virgule, alors il a un seul thème et nous avons juste à l'ajouter.
#             themes_finaux.append(themes_uniques[i])
#     return themes_finaux

#Fonction servant à enlever les caractères 'parasites' et enlever les doublons
def nettoyage(themes_extrait):
    #Création de la liste de sortie
    themes_nettoyes=[]
    #Pour chaque champ, on enlève les caractères 'parasite', on ajoute le résultat dans la liste de sortie
    for i in range(0,len(themes_extrait)):
        if ',' in themes_extrait[i]:
            for j in themes_extrait[i].split(','):
                themes_nettoyes.append(j.strip(" '\"[]#"))
        else:
            themes_nettoyes.append(themes_extrait[i].strip(" '\"[]#"))
    #On enlève les doublons
    themes_nettoyes=list(np.unique(themes_nettoyes))
    themes_nettoyes.remove('')
    return themes_nettoyes

#Fonction principale n'ayant pas besoin d'entrée, elle permettra d'avoir la liste des thèmes.
#Cette fonction utilise les fonctions extraction (qui utilise la fonction separation) et nettoyage.
def traitement_themes():
    #Import des la liste des thèmes
    themes=list(articles['art_tag'].dropna())
    #On enlève les doublons
#     themes_uniques=list(np.unique(themes))
#     #Appel de la fonction extraction
#     themes_extrait=extraction(themes_uniques)
    #Appel de la fonction nettoyage
    themes_finaux=nettoyage(themes)
    #Retourne les thèmes
    return themes_finaux

#Fonction servant à calculer le nb d'articles par thèmes
def nbarticles_themes():
    #Appel traitements_thèmes
    themes=articles['art_tag'].dropna()
    l_themes = traitement_themes()
    #Création de la liste de sortie, tous ses champs doivent être initialisés à 0
    nb_articles=[0]*len(l_themes)
    #Boucle pour traiter tous les thèmes
    for i in range(0,len(l_themes)):
        #Boucle pour traiter tous les articles
        for j in range (0,len(themes)):
            #Indendation simple, si le thème correspond à l'article
            if l_themes[i] in themes[j]:
                nb_articles[i]=nb_articles[i]+1
    return nb_articles

#Fonction créeant un dataframe sur les articles par thèmes
def dataframe_themes():
    l_themes=traitement_themes()
    l_nb_articles=nbarticles_themes()
    df = pd.DataFrame(l_nb_articles, index = l_themes, columns = ['nb_articles'])
    return df

def get_cat_options(articles):
    list_cat=traitement_themes()
    cat_options=[]
    for n in range(0,len(list_cat)):
        dict={'label':list_cat[n],'value':list_cat[n]}
        cat_options.append(dict)
    return cat_options

print(get_cat_options(articles))

def get_min_max_dates(articles):
    date = articles[['art_date']]
    min_date = min(date['art_date'].dropna())
    max_date = dt.today()
    return (min_date,max_date)



#graphs
grp_languages = articles.groupby('art_lang')[["art_title"]].count().reset_index().rename(columns={'art_title':'nb_articles'})
fig_lang = px.pie(grp_languages, values='nb_articles', names='art_lang', title='Languages')

df_themes=dataframe_themes()
df_themes['nom_themes']=df_themes.index
fig_themes = px.histogram(df_themes,x='nom_themes', y='nb_articles')

[{'label': '13 octobre 2020', 'value': '13 octobre 2020'}, {'label': '3 G', 'value': '3 G'}, {'label': '3D', 'value': '3D'}, {'label': '5G', 'value': '5G'}, {'label': 'A la une', 'value': 'A la une'}, {'label': 'ACPUSI', 'value': 'ACPUSI'}, {'label': 'ADGCF', 'value': 'ADGCF'}, {'label': 'ADN Ouest', 'value': 'ADN Ouest'}, {'label': 'ANDCDG', 'value': 'ANDCDG'}, {'label': 'AP Connect', 'value': 'AP Connect'}, {'label': 'Accompagnement', 'value': 'Accompagnement'}, {'label': 'Acteurs des territoires', 'value': 'Acteurs des territoires'}, {'label': 'Action Coeur de Ville', 'value': 'Action Coeur de Ville'}, {'label': 'Action logement', 'value': 'Action logement'}, {'label': 'Actualités', 'value': 'Actualités'}, {'label': 'Actualités Financières', 'value': 'Actualités Financières'}, {'label': 'Administration', 'value': 'Administration'}, {'label': "Administration - Réforme de l'Etat", 'value': "Administration - Réforme de l'Etat"}, {'label': 'Administration centrale', 'value': 'Administra

KeyError: 0

In [261]:
# Modules d'affichage

def affichage_card(liste_art,l_index=[],df):
    for nb in liste_art:
        l_index.append(html.Div([
            dbc.Col(html.Div([
                dbc.Card(
                    dbc.CardBody(
                        [
                            dbc.CardLink(html.H5(df[df['art_id']=nb]['art_title'], className="card-title",style = {"width":"auto","align" : "justify","text-align": "justify"}), href="/page-"+df['art_id'],target='blank'),
                            html.P(
                                html.P(" ".join(df['art_content'][nb].split(' ')[:50]),style = {"width":"auto","align" : "justify","text-align": "justify"})
                            ),
                        ]
                    ),
                    className="w-100 mb-3",
                ),
            ])
            )
        ]))
    return l_index



SyntaxError: invalid syntax (<ipython-input-261-42431634bb19>, line 10)

In [269]:
app = JupyterDash(__name__,external_stylesheets=[dbc.themes.BOOTSTRAP],suppress_callback_exceptions=True)

#Intégrer la barre de recherche avec bouton de sélection
search_bar = dbc.Row(
    [
        dbc.Col(dcc.Input(id='username',type="search", placeholder="Search")),
        dbc.Col(html.Button(id='submit-button', type='submit', children='Submit', style=button_style))],
    no_gutters=True,
    className="ml-auto flex-nowrap mt-3 mt-md-0",
    align="center")

#Barre de navigation
nav_bar = dbc.Navbar(
    [
        html.A(
            # Use row and col to control vertical alignment of logo / brand
            dbc.Row(
                [
                    dbc.Col(dbc.NavbarBrand("Home", className="ml-2")),
                ],
                align="center",
                no_gutters=True,
            ),
            href="/",
        ),
        dbc.NavbarToggler(id="navbar-toggler"),
        dbc.Collapse(search_bar, id="navbar-collapse", navbar=True),
    ],
    style = {"color":"Dark"},
    dark=True)


#Création des filtres
side_bar = html.Div(
    [
        html.H2("Filters"),
        html.Hr(),
        html.Div([
        dbc.Col(
        [
            # -------TOUS LES DROPDOWNS + DATE PICKER ---------
            # gerer fonction du boutton voir components dash (tuto)
            dbc.Col(html.Div([html.Label('Language'),
                              dcc.Dropdown(id='lang_filt',
                    options=get_lang_options(articles),
                    multi=True
                    )]), width='auto'),
            dbc.Col(html.Div([html.Label('Tag'),
                              dcc.Dropdown(id='tag_filt',
                    options=get_cat_options(articles),
                    multi=True
                    )]), width='auto'),
            dbc.Col(html.Div([html.Label('Date Range'), dcc.DatePickerRange(
                    id='date-picker-range',
                    start_date=get_min_max_dates(articles)[0],
                    end_date=get_min_max_dates(articles)[1]
                    # end_date_placeholder_text='Select a date!'
                    )]), width='auto'),
            dbc.Col(html.Div([html.Label('Number of article'), dcc.Input(
                    id = 'number',
                    type= "number",
                    value = 10
                    )])),
            html.Hr(),
            dbc.Col(html.Div([html.Label('Filters'),
                              html.Br(),
                              html.Button('Refresh Filters',
                                          id='reset_button', style=button_style)]))
        ]),
        ]),
    ],
    style=SIDEBAR_STYLE,
)


# Modules d'affichage

def affichage_card(liste_art,l_index=[]):   
    """
    Input:
        liste_art : list of article index in int
        l_index : list of html.Div for display
    Output:
        l_index : list of html.Div for display with the articles cards
    """
    for nb in liste_art:
        # in the index of selected articles
        l_index.append(html.Div([
            dbc.Col(html.Div([
                dbc.Card(
                    dbc.CardBody(
                        [
                            dbc.CardLink(html.H5(articles['art_title'][nb], className="card-title",style = {"width":"auto","align" : "justify","text-align": "justify"}), href="/page-"+str(nb),target='blank'),
                            html.P(
                                html.P(" ".join(articles['art_content'][nb].split(' ')[:50]),style = {"width":"auto","align" : "justify","text-align": "justify"})
                            ),
                        ]
                    ),
                    className="w-100 mb-3",
                ),
            ])
            )
        ]))
    return l_index


# articles display page
card = html.Div(id ='out_card')

@app.callback(
    Output('out_card', 'children'),
    [Input('number', 'value')],
    [Input('submit-button', 'n_clicks')],
    [dash.dependencies.Input('date-picker-range', 'start_date'),
     dash.dependencies.Input('date-picker-range', 'end_date')],
    [State('username', 'value')],
)

def update_card_div(input_value2,clicks,start_date,end_date,input_value):
    
    filtered_df = articles[:input_value2]

    cards = affichage_card(range(len(filtered_df)))

    cards += [html.Div([dbc.Button("Return to top", color="primary", className="mr-1",href='#',style= button_style)])]
    
    if start_date is not None and end_date is not None:
        filtre_date = []
        page_date = []
#         start_date_object = date.fromisoformat(start_date)
#         end_date_object = date.fromisoformat(end_date)
        for nb in range(len(filtered_df)):
            if articles['art_date'][nb]>start_date and articles['art_date'][nb]<end_date:
                filtre_date.append(nb)
        if filtre_date != []:
            page_date.append(html.Div([html.H2("Article(s) trouvé(s) : " + str(len(filtre_date))),
                                         html.Br()]))
            page_date = affichage_card(filtre_date, page_date)
            page_date += [html.Div([dbc.Button("Return to top", color="primary", className="mr-1",href='#'),
                                     dbc.Button("Return", color="primary", className="mr-1",href='/')])]
        else:
            page_date.append(html.Div([html.H2("Aucun article trouvé")]))
        return page_date
#         start_date_string = start_date_object.strftime('%B %d, %Y')
#         end_date_string = end_date_object.strftime('%B %d, %Y')
    else:
        return cards
    
    if clicks is not None:
        element = []
        search_page = []
        
        if input_value == '':
            return cards
        
        for nb in range(len(filtered_df)):
            t = input_value
            if t.lower() in filtered_df['art_title'][nb].lower() or t.lower() in filtered_df['art_content'][nb].lower() or t.lower() in filtered_df['art_tag'][nb].lower():
                element.append(nb)
                range(len(filtered_df))
        if element != []:
            search_page.append(html.Div([html.H2("Article(s) trouvé(s) : " + str(len(element))),
                                         html.Br()]))
            search_page = affichage_card(element, search_page)
            search_page += [html.Div([dbc.Button("Return to top", color="primary", className="mr-1",href='#'),
                                     dbc.Button("Return", color="primary", className="mr-1",href='/')])]
        else:
            search_page.append(html.Div([html.H2("Aucun article trouvé")]))
        return search_page
    else:
        return cards

# @app.callback(Output('out_card','children'),
#              [dash.dependencies.Input('tag_filt', 'value')],
#              )   
# def filtered_card(input_value):
#     element=[]
#     for nb in len(range(articles)):
#         if input_value in articles['art_tag']:
#             element.append(nb)
#     if element != []:
#         search_page.append(html.Div([html.H2("Article(s) trouvé(s) : " + str(len(element))),
#                                          html.Br()]))
#         search_page = affichage_card(element, search_page)
#         search_page += [html.Div([dbc.Button("Return to top", color="primary", className="mr-1",href='#'),
#                                      dbc.Button("Return", color="primary", className="mr-1",href='/')])]
#     else:
#         search_page.append(html.Div([html.H2("Aucun article trouvé")]))
#     return search_page
    

# single article page
article = html.Div(id ='out_article')

@app.callback(
    Output("out_article", "children"),
    [dash.dependencies.Input('url', 'pathname')]
)

def toggle_article(pathname):
        
    index = int(pathname.split("-")[1])

    df_article = articles.iloc[index]

    article = html.Div([
        dbc.Col(html.Div([
            dbc.Button("Return", color="primary", className="mr-1",href='/'),
            html.Hr(),
            dbc.Card(
                dbc.CardBody(
                    [
                        
                        dbc.CardLink(html.H5(df_article['art_title'], className="card-title",style = {"width":"auto","align" : "justify","text-align": "justify"}), href=df_article['art_url']),
                        html.P(df_article['art_content'],style={"text-align": "justify"}),
#                         html.Button('Return',id='return',href='http://127.0.0.1:8060/')
                        dbc.Button("Return to top", color="primary", className="mr-1",href='#')
                    ]
                ),
                className="w-100 mb-3"
            ),
        ])
    )])


    return article


# add callback for toggling the collapse on small screens
@app.callback(
    Output("navbar-collapse", "is_open"),
    [Input("navbar-toggler", "n_clicks")],
    [State("navbar-collapse", "is_open")],
)
def toggle_navbar_collapse(n, is_open):
    if n:
        return not is_open
    return is_open


content = html.Div([card], id="page-content", style=CONTENT_STYLE)

#Crétion des onglets
app.layout = html.Div([html.Div([
    dcc.Tabs(id='tabs-example', value='tab-1', children=[
        dcc.Tab(label='Articles', value='tab-1',style=tab_style, selected_style=tab_select_style),
        dcc.Tab(label='Statistiques', value='tab-2',style=tab_style, selected_style=tab_select_style),
    ]),
    html.Div(id='tabs-example-content')
])
])

onglet = html.Div(
    [
        nav_bar,
        html.Hr(),
        dbc.Row(
            [
                dbc.Col([dcc.Location(id="url"),side_bar], width = "auto"),
                dbc.Col(content, style={"width":"auto","display": "inline-block"}),
                
            ],
        )
    ]
)

#Affichage du filtres que dans l'onglet article 
@app.callback(Output('tabs-example-content', 'children'),
              [Input('tabs-example', 'value')])
def render_content(tab):
    if tab == 'tab-1':
        return [onglet]

    elif tab == 'tab-2':
        return html.Div([
            html.Hr(),
            dbc.Row([
                dbc.Col([dcc.Location(id="url"),side_bar], width = "auto"),
                dbc.Col(
                    dcc.Graph(figure = fig_lang),
                    width={'size':3}),
                dbc.Col(
                    dcc.Graph(figure = fig_themes),
                    width={'size':5})],),])

    
#Affichage des articles dans l'onglet Article
@app.callback(dash.dependencies.Output('page-content', 'children'),
             [dash.dependencies.Input('url', 'pathname')])


def render_page_content(pathname):
    if pathname == "/":
        return content
    elif "/page-" in pathname:
        return article
    # If the user tries to reach a different page, return a 404 message
    return dbc.Jumbotron(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognised..."),
        ]
    )

# Callback for reset filter 
@app.callback(
    Output('lang_filt', 'value'),
    Output('tag_filt', 'value'),
    Output('date-picker-range', 'start_date'),
    Output('date-picker-range', 'end_date'),
    Input('reset_button','n_clicks')
)
def reset_filter(n_click):
    data = articles
    bornes = get_min_max_dates(data)
    return [],[],bornes[0],bornes[1]

if __name__ == "__main__":
    app.run_server(debug=True,port=8061)


Dash app running on http://127.0.0.1:8061/
